In [26]:
using QuantumCumulants
using IterTools
using Symbolics
using SymbolicUtils
using SymbolicUtils.Rewriters
using OrdinaryDiffEq, ModelingToolkit
using Plots

include("../src/diagrams.jl")
include("../src/contractions.jl")
include("../src/lindblad.jl")
include("../src/printing.jl")
include("../src/poles.jl")

reshape_sols

In [2]:
node = DiagramNode((1,2))
ds = get_diagrams(node)

4-element Vector{Any}:
 [(1, 2)]
 [(0, 1), (1, 1)]
 [(0, 2), (1, 0)]
 [(0, 1), (0, 1), (1, 0)]

In [33]:
h_cav = FockSpace(:cavity)
h_atom = NLevelSpace(:atom, (:g,:e))
h = tensor(h_cav, h_atom)

@qnumbers a::Destroy(h) σ::Transition(h)
@cnumbers ω_0 ω_d g κ γ ϵ ω_3 ω_1

#σz = 2* σ(:e, :e)  - 1
#σx = σ(:g, :e) + σ(:e, :g)
#σy = 1im* (σ(:e, :g) - σ(:g, :e))
@syms t::Real
H_rab = ω_0 * a'*a + ω_d/2 * σz + g*(a' + a)*(σx) + ϵ*(a*exp(1im*ω_3*t) + a'*exp(-1im*ω_3*t))
#@register f(t)
ωs = [0,0,0,ω_3, -ω_3]
hs = [ω_0 * a'*a,ω_d/2 * σz,g*(a' + a)*(σx),ϵ*a, ϵ*a']

ωs2 = [0,0,ω_3,-ω_3]
hs2 = [0.5*ω_0 * a'*a,0.5*ω_0 * a'*a,ϵ*a, ϵ*a']

4-element Vector{QuantumCumulants.QMul{Nothing}}:
 0.5ω_0*(a′*a)
 0.5ω_0*(a′*a)
 ϵ*(a)
 ϵ*(a′)

In [14]:
k=1
ω_list = repeated_combinations(ωs2, k)
h_list = repeated_combinations(hs2, k)
print(h_list)

Vector{QuantumCumulants.QMul{Nothing}}[[0.5

ω_0*(a′*a)], [0.5ω_0*(a′*a)], [ϵ*(a)], [ϵ*(a′)]]

In [37]:
@cnumbers ω_1 ω_2 ω_3
#simplify(contraction_coeff((3,0),[ω_1,-ω_1,ω_2]))
c, cs = contraction_coeff((3,0),[0,ω_3,-ω_3])
c

(-12.0 - 3.0ω_3*(τ^2)) / (-ω_3) + 0.5 / (-ω_3) + (-(exp(-0.5(τ^2)*(ω_3^2))^2)) / (ω_3^2) + (36.0(6.0 + -1 / ω_3 - ω_3*(τ^2))*(exp(-0.5(τ^2)*(ω_3^2))^2)) / ω_3

In [34]:
k=2
h_eff = effective_hamiltonian(k, ωs2, hs2)
H_eff = simplify(h_eff)
secondOrderRule = @acrule((~a)*(~y)*(~z) + (~b)*(~y)*(~z) => ((~a)+(~b))*(~y)*(~z))
firstOrderRule = @acrule((~a)*(~y) + (~b)*(~y) => ((~a)+(~b))*(~y))
rules = Chain([firstOrderRule,secondOrderRule])#, addRule])
H1 = Fixpoint(rules)(H_eff)

(ϵ*exp((0 + 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2)) + (ω_0*ϵ*exp(-0.5(τ^2)*(ω_3^2))*exp((0 + 1im)*t*ω_3)) / ω_3 + (4.5 + 0.0im)ω_0*ϵ*exp(-0.5(τ^2)*(ω_3^2))*exp((0 + 1im)*t*ω_3)*(a)+ϵ*exp((0 - 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2)) + (ω_0*ϵ*exp(-0.5(τ^2)*(ω_3^2))*exp((0 - 1im)*t*ω_3)) / ω_3 + (4.5 + 0.0im)ω_0*ϵ*exp(-0.5(τ^2)*(ω_3^2))*exp((0 - 1im)*t*ω_3)*(a′)+ω_0 + (9.0 + 0.0im)(ω_0^2)*(a′*a)+(ϵ^2) / ω_3+((ϵ^2)*exp(-(τ^2)*(ω_3^2))) / ω_3+(9.0 + 0.0im)(ω_0^2)*(a′*a′*a*a)+(9.0 + 0.0im)ω_0*ϵ*exp(-0.5(τ^2)*(ω_3^2))*exp((0 - 1im)*t*ω_3)*(a′*a′*a)+(9.0 + 0.0im)ω_0*ϵ*exp(-0.5(τ^2)*(ω_3^2))*exp((0 + 1im)*t*ω_3)*(a′*a*a))

In [38]:
eqs = meanfield([a] , H1; order=2)
eqs = complete(eqs)

∂ₜ(⟨a⟩) = ((0.0 - 1.0im)*ω_0*ω_3*⟨a⟩ + (0.0 - 1.0im)*ω_0*ϵ*exp((0 - 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2)) + (0.0 - 1.0im)*ω_3*ϵ*exp((0 - 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2)) + (0.0 - 18.0im)*ω_3*(⟨a′⟩*⟨a*a⟩ + 2⟨a⟩*⟨a′*a⟩ - 2(⟨a⟩^2)*⟨a′⟩)*(ω_0^2) + (0.0 - 9.0im)*ω_3*(ω_0^2)*⟨a⟩ + (0.0 - 4.5im)*ω_0*ω_3*ϵ*exp((0 - 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2)) + (0.0 - 9.0im)*ω_0*ω_3*ϵ*exp((0 + 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2))*⟨a*a⟩ + (0.0 - 18.0im)*ω_0*ω_3*ϵ*exp((0 - 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2))*⟨a′*a⟩) / ω_3
∂ₜ(⟨a*a⟩) = ((0.0 - 36.0im)*ω_3*((3⟨a⟩*⟨a*a⟩ - 2(⟨a⟩^3))*⟨a′⟩ + 3(⟨a′⟩*⟨a*a⟩ + 2⟨a⟩*⟨a′*a⟩ - 2(⟨a⟩^2)*⟨a′⟩)*⟨a⟩ + 3⟨a′*a⟩*⟨a*a⟩ + 6(⟨a⟩^3)*⟨a′⟩ - 6(⟨a⟩^2)*⟨a′*a⟩ - 6⟨a′⟩*⟨a⟩*⟨a*a⟩)*(ω_0^2) + (0.0 - 2.0im)*ω_0*ω_3*⟨a*a⟩ + (0.0 - 36.0im)*ω_3*(ω_0^2)*⟨a*a⟩ + (0.0 - 2.0im)*ω_0*ϵ*⟨a⟩*exp((0 - 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2)) + (0.0 - 2.0im)*ω_3*ϵ*⟨a⟩*exp((0 - 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2)) + (0.0 - 18.0im)*ω_0*ω_3*ϵ*(3⟨a⟩*⟨a*a⟩ - 2(⟨a⟩^3))*exp((0 + 1im)*t*ω_3 - 0.5(τ^2)*(ω_3^2)) + (0.0 - 27.0im)*ω_0*ω_3*ϵ*⟨a⟩*exp((0 -

In [28]:
#H_test = σ(1,2,2)⊗σ(2,1,1) #9*(ω_1*σ(2,2,1)+ω_2*σ(2,2,2) - g*σ(1,2,1)*σ(2,1,2))# -  g*σ(1,2,1)*σ(2,2,1) 
            #-  g*σ(1,1,2)*σ(2,1,2)- g*σ(1,1,2)*σ(2,2,1)) - 4.5*(ω_1 + ω_2)
            
#H_test2 = 9*ω_1*σ(2,2,1) - 4.5*ω_1 + ϵ_d*exp(1im*ω_d)/(2*ω_d^2)  +σ(1,2,1) + σ(2,1,1) # (ϵ_d/(2*ω_d^2))*σ(1,2,1) #+ (ϵ_d/(2*ω_d^2))*σ(2,1,1)
@register f(t)
#f(t) = exp(1im*ω_d*t)
H_rab = ω_0 * a'*a + ω_d/2 * σz + g*(a' + a)*(σx) + f(t)*(a + a')


(ω_d*(σee)+(-1//2)*ω_d+ω_0*(a′*a)+g*(a′*σge)+g*(a′*σeg)+g*(a*σge)+g*(a*σeg)+f(t)*(a)+f(t)*(a′))

In [33]:
eqs = meanfield([a] , H_rab; order=3, iv=t)
eqs = complete(eqs)

∂ₜ(⟨a⟩) = (0 - 1im)*f(t) + g*((0 - 1im)*⟨σeg⟩ + (0 - 1im)*⟨σge⟩) + (0 - 1im)*ω_0*⟨a⟩
∂ₜ(⟨σeg⟩) = g*((0 + 1im)*⟨a′⟩ + (0 + 1im)*⟨a⟩ + (0 - 2im)*⟨a′*σee⟩ + (0 - 2im)*⟨a*σee⟩) + (0 + 1im)*ω_d*⟨σeg⟩
∂ₜ(⟨a′*σee⟩) = g*((0 + 1im)*⟨σge⟩ + (0 - 1im)*⟨a′*a′*σeg⟩ + (0 + 1im)*⟨a′*a′*σge⟩ + (0 - 1im)*⟨a′*a*σeg⟩) + (0 + 1im)*g*⟨a′*a*σge⟩ + (0 + 1im)*ω_0*⟨a′*σee⟩ + (0 + 1im)*f(t)*⟨σee⟩
∂ₜ(⟨a′*a′*σeg⟩) = g*((0 + 1im)*⟨a′*a′*a′⟩ + (0 - 2im)*⟨a′*σee⟩ + (0 + 1im)*⟨a′*a′*a⟩ + (0 - 2im)*⟨σee⟩*⟨a′*a′*a⟩ + (0 + 4im)*(⟨a′⟩^2)*⟨a*σee⟩ + (0 - 4im)*⟨a′⟩*⟨a′*a*σee⟩ + (0 - 2im)*⟨a⟩*⟨a′*a′*σee⟩ + (0 - 2im)*⟨a′*a′⟩*⟨a*σee⟩ + (0 - 4im)*⟨a′*a⟩*⟨a′*σee⟩ + (0 + 8im)*⟨a′⟩*⟨a⟩*⟨a′*σee⟩ + (0 + 4im)*⟨a⟩*⟨σee⟩*⟨a′*a′⟩ + (0 + 8im)*⟨a′⟩*⟨σee⟩*⟨a′*a⟩ + (0 - 12im)*(⟨a′⟩^2)*⟨a⟩*⟨σee⟩) + (0 - 2im)*g*(⟨σee⟩*⟨a′*a′*a′⟩ + 3⟨a′*a′⟩*⟨a′*σee⟩ + 3⟨a′⟩*⟨a′*a′*σee⟩ + 6(⟨a′⟩^3)*⟨σee⟩ - 6(⟨a′⟩^2)*⟨a′*σee⟩ - 6⟨a′⟩*⟨σee⟩*⟨a′*a′⟩) + (0 + 2im)*g*⟨a′⟩ + (0 + 2im)*ω_0*⟨a′*a′*σeg⟩ + (0 + 2im)*f(t)*⟨a′*σeg⟩ + (0 + 1im)*ω_d*⟨a′*a′*σeg⟩
∂ₜ(⟨a′*a′*σge⟩

In [34]:
substitute(eqs, Dict([f(t) => exp(1im*ω_d*t)]))

∂ₜ(⟨a⟩) = (0 - 1im)*exp((0 + 1im)*t*ω_d) + g*((0 - 1im)*⟨σeg⟩ + (0 - 1im)*⟨σge⟩) + (0 - 1im)*ω_0*⟨a⟩
∂ₜ(⟨σeg⟩) = g*((0 + 1im)*⟨a′⟩ + (0 + 1im)*⟨a⟩ + (0 - 2im)*⟨a′*σee⟩ + (0 - 2im)*⟨a*σee⟩) + (0 + 1im)*ω_d*⟨σeg⟩
∂ₜ(⟨a′*σee⟩) = g*((0 + 1im)*⟨σge⟩ + (0 - 1im)*⟨a′*a′*σeg⟩ + (0 + 1im)*⟨a′*a′*σge⟩ + (0 - 1im)*⟨a′*a*σeg⟩) + (0 + 1im)*g*⟨a′*a*σge⟩ + (0 + 1im)*ω_0*⟨a′*σee⟩ + (0 + 1im)*⟨σee⟩*exp((0 + 1im)*t*ω_d)
∂ₜ(⟨a′*a′*σeg⟩) = g*((0 + 1im)*⟨a′*a′*a′⟩ + (0 - 2im)*⟨a′*σee⟩ + (0 + 1im)*⟨a′*a′*a⟩ + (0 - 2im)*⟨σee⟩*⟨a′*a′*a⟩ + (0 + 4im)*(⟨a′⟩^2)*⟨a*σee⟩ + (0 - 4im)*⟨a′⟩*⟨a′*a*σee⟩ + (0 - 2im)*⟨a⟩*⟨a′*a′*σee⟩ + (0 - 2im)*⟨a′*a′⟩*⟨a*σee⟩ + (0 - 4im)*⟨a′*a⟩*⟨a′*σee⟩ + (0 + 8im)*⟨a′⟩*⟨a⟩*⟨a′*σee⟩ + (0 + 4im)*⟨a⟩*⟨σee⟩*⟨a′*a′⟩ + (0 + 8im)*⟨a′⟩*⟨σee⟩*⟨a′*a⟩ + (0 - 12im)*(⟨a′⟩^2)*⟨a⟩*⟨σee⟩) + (0 - 2im)*g*(⟨σee⟩*⟨a′*a′*a′⟩ + 6(⟨a′⟩^3)*⟨σee⟩ + 3⟨a′*a′⟩*⟨a′*σee⟩ + 3⟨a′⟩*⟨a′*a′*σee⟩ - 6(⟨a′⟩^2)*⟨a′*σee⟩ - 6⟨a′⟩*⟨σee⟩*⟨a′*a′⟩) + (0 + 2im)*g*⟨a′⟩ + (0 + 2im)*ω_0*⟨a′*a′*σeg⟩ + (0 + 2im)*⟨a′*σeg⟩*exp((0 + 1im)*t

In [49]:
k=1
γ, J_ops, Jd_ops = effective_dissipator(k, ωs, hs,t)
#print(5)

(Any[], Any[], Any[])

In [26]:
f(t) = exp(1im*ω_d*t)
@named sys = ODESystem(eqs)
#ω_0,ω_d, g = 1,1,1
vals = [1.0;1.0;1.0]
u0 = zeros(ComplexF64, length(eqs))
prob = ODEProblem(sys,u0,(0.0, 1), [g; ω_0; ω_d].=>vals)
sol = solve(prob,Tsit5(),maxiters=1e7)

ArgumentError: ArgumentError: Sym{Parameter, Base.ImmutableDict{DataType, Any}}[g, ω_0, ω_d] are missing from the variable map.

In [24]:
length(Jd_ops)

275

In [39]:
#Get meanfield equations using QuantumCumulants
eqs = meanfield([σz1, σz2, σx1, σx2, σx1*σz2] , H1; order=2,iv=t)

∂ₜ(1 - 2⟨σ_{1}22⟩) = ((0.0 - 4608.0im)*g*(ω_d^8)*⟨σ_{1}21*σ_{2}12⟩ + (0.0 - 4608.0im)*g*(ω_d^8)*⟨σ_{1}21*σ_{2}21⟩ + (0.0 + 4608.0im)*g*(ω_d^8)*⟨σ_{1}12*σ_{2}12⟩ + (0.0 + 4608.0im)*g*(ω_d^8)*⟨σ_{1}12*σ_{2}21⟩ + (0.0 - 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}21⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}21⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}12⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}12⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}21⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}21⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}12⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}12⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 3072.0im)*g*ϵ_d*(τ^2)*(ω_d^8)*⟨σ_{1}21⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 3072.0im)*g*ϵ_d*(τ^2)*

In [28]:
eqs = complete(eqs)

MethodError: MethodError: Cannot `convert` an object of type SymbolicUtils.Term{QuantumCumulants.AvgSym, Nothing} to an object of type SymbolicUtils.Add{CNumber, Int64, Dict{Any, Number}, Nothing}
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/essentials.jl:218
  SymbolicUtils.Add{X, T, D, M}(::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) where {X<:Number, T<:Number, D, M} at ~/.julia/packages/SymbolicUtils/qulQp/src/types.jl:617

In [40]:
H_test = ((9.0)ω_1*(σ_{1}22)+(-4.5 + 0.0im)ω_1 + (-4.5 + 0.0im)ω_2+(9.0 + 0.0im)ω_2*(σ_{2}22)+(-9.0 - 0.0im)g*(σ_{1}21*σ_{2}12)+(-9.0 - 0.0im)g*(σ_{1}21*σ_{2}21)+(-9.0 - 0.0im)g*(σ_{1}12*σ_{2}12)+(-9.0 - 0.0im)g*(σ_{1}12*σ_{2}21))

ErrorException: syntax: missing comma or ) in argument list